In [2]:

%pip install requests_cache -q
%pip install tqdm
%pip install pandas
%pip install python-dotenv
%pip install requests
%pip install kalshi-python


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note

In [3]:
import requests
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("KALSHI_KEY")

event_ticker = 'KXLIVENATIONUS-30'
KALSHI_EVENTS = "https://api.elections.kalshi.com/trade-api/v2/events"
KALSHI_EVENT = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}"

In [4]:
r = requests.get(KALSHI_EVENTS)
response = r.json()
response

{'events': [{'event_ticker': 'KXROBOTMARS-35',
   'series_ticker': 'KXROBOTMARS',
   'sub_title': 'Before 2035',
   'title': 'Will a humanoid robot walk on Mars before a human does?',
   'collateral_return_type': '',
   'mutually_exclusive': False,
   'category': 'Science and Technology'},
  {'event_ticker': 'KXNEXTPOPE-35',
   'series_ticker': 'KXNEXTPOPE',
   'sub_title': 'Before 2035',
   'title': 'Who will the next Pope be?',
   'collateral_return_type': 'MECNET',
   'mutually_exclusive': True,
   'category': 'World'},
  {'event_ticker': 'KXLALEADEROUT-35',
   'series_ticker': 'KXLALEADEROUT',
   'sub_title': 'Before 2035',
   'title': 'Which of these Latin America leaders will leave office next?',
   'collateral_return_type': 'MECNET',
   'mutually_exclusive': True,
   'category': 'Politics'},
  {'event_ticker': 'KXG7LEADEROUT-35',
   'series_ticker': 'KXG7LEADEROUT',
   'sub_title': 'Before 2035',
   'title': 'Which of these G7 leaders will leave office next?',
   'collateral_ret

In [7]:
import pandas as pd
from tqdm import tqdm
import pickle

# Read Kalshi Events to a DataFrame
cursor = ""
events = []

with tqdm(total=float('inf'), desc="Fetching markets") as pbar:
    while True:
        url = KALSHI_EVENTS

        response = requests.get(url, params=dict(
            limit=200,
            cursor=cursor,
            with_nested_markets=True
        ))
        r = response.json()

        # Append the new events
        events.extend(r.get("events", []))
        
        # Update cursor and break if there are no new results
        new_cursor = r.get("cursor")
        if new_cursor == cursor or not new_cursor:
            break
        cursor = new_cursor
        pbar.update(len(r.get("events", [])))

# Convert to DataFrame
events_df = pd.DataFrame(events)

# Cache into a pickle file
with open('../data/kalshi_event_data.pkl', 'wb') as f:
    pickle.dump(events_df, f)

Fetching markets: 3600it [00:18, 196.57it/s]


In [9]:
import pickle

with open('../data/kalshi_event_data.pkl', 'rb') as f:
    df = pickle.load(f)
df

,event_ticker,series_ticker,sub_title,title,collateral_return_type,mutually_exclusive,category,markets,strike_date,strike_period
0,KXROBOTMARS-35,KXROBOTMARS,Before 2035,Will a humanoid robot walk on Mars before a hu...,,False,Science and Technology,"[{'ticker': 'KXROBOTMARS-35', 'event_ticker': ...",NaN,NaN
1,KXNEXTPOPE-35,KXNEXTPOPE,Before 2035,Who will the next Pope be?,MECNET,True,World,"[{'ticker': 'KXNEXTPOPE-35-GCYP', 'event_ticke...",NaN,NaN
2,KXLALEADEROUT-35,KXLALEADEROUT,Before 2035,Which of these Latin America leaders will leav...,MECNET,True,Politics,"[{'ticker': 'KXLALEADEROUT-35-NM', 'event_tick...",NaN,NaN
3,KXG7LEADEROUT-35,KXG7LEADEROUT,Before 2035,Which of these G7 leaders will leave office next?,MECNET,True,Politics,"[{'ticker': 'KXG7LEADEROUT-35-SI', 'event_tick...",NaN,NaN
4,KXEULEADEROUT-35,KXEULEADEROUT,Before 2035,Which of these European leaders will leave off...,MECNET,True,Politics,"[{'ticker': 'KXEULEADEROUT-35-UK', 'event_tick...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3595,KXUSDJPYH-25APR2314,KXUSDJPYH,"Apr 23, 2025 at 2pm EDT","USD/JPY price on Apr 23, 2025 at 2pm EDT?",DIRECNET,False,Financials,"[{'ticker': 'KXUSDJPYH-25APR2314-T146.85900', ...",2025-04-23T18:00:00Z,NaN
3596,KXNASDAQ100U-25APR23H1400,KXNASDAQ100U,"On Apr 23, 2025 at 2pm EDT","Nasdaq price on Apr 23, 2025 at 2pm EDT?",DIRECNET,False,Financials,[{'ticker': 'KXNASDAQ100U-25APR23H1400-T19789....,2025-04-23T18:00:00Z,NaN
3597,KXINXU-25APR23H1400,KXINXU,"On Apr 23, 2025 at 2pm EDT","S&P price on Apr 23, 2025 at 2pm EDT?",DIRECNET,False,Financials,"[{'ticker': 'KXINXU-25APR23H1400-T6144.9999', ...",2025-04-23T18:00:00Z,NaN
3598,KXEURUSDH-25APR2314,KXEURUSDH,"Apr 23, 2025 at 2pm EDT","EUR/USD price on Apr 23, 2025 at 2pm EDT?",DIRECNET,False,Financials,"[{'ticker': 'KXEURUSDH-25APR2314-T1.17259', 'e...",2025-04-23T18:00:00Z,NaN


In [10]:
result = df[df['category'] == 'Elections']

# Display the result
result

,event_ticker,series_ticker,sub_title,title,collateral_return_type,mutually_exclusive,category,markets,strike_date,strike_period
5,KXBRUVSEAT-35,KXBRUVSEAT,In the next U.K. general election,Will Andrew Tate's party win a seat in the nex...,,False,Elections,"[{'ticker': 'KXBRUVSEAT-35', 'event_ticker': '...",NaN,NaN
20,KXROMANIATOP-30,KXROMANIATOP,In the next first round,Who will be 1st in the Romanian election first...,MECNET,True,Elections,"[{'ticker': 'KXROMANIATOP-30-NDAN', 'event_tic...",NaN,NaN
43,KXUKPARTY-29,KXUKPARTY,In the next U.K. election,Which party will win the next U.K. election?,MECNET,True,Elections,"[{'ticker': 'KXUKPARTY-29-BRUV', 'event_ticker...",NaN,NaN
54,KXPRESPARTY-2028,KXPRESPARTY,In 2028,Which party will win the 2028 Presidential Ele...,MECNET,True,Elections,"[{'ticker': 'KXPRESPARTY-2028-R', 'event_ticke...",NaN,NaN
94,KXCAPCONTROL-29,KXCAPCONTROL,During Trump's term,Will Trump impose capital controls?,,False,Elections,"[{'ticker': 'KXCAPCONTROL-29', 'event_ticker':...",NaN,NaN
119,KXVPRESNOMR-28,KXVPRESNOMR,In 2028,Who will be the Republican VP nominee in 2028?,MECNET,True,Elections,"[{'ticker': 'KXVPRESNOMR-28-TC', 'event_ticker...",NaN,NaN
120,KXVPRESNOMD-28,KXVPRESNOMD,In 2028,Who will be the Democratic VP nominee in 2028?,MECNET,True,Elections,"[{'ticker': 'KXVPRESNOMD-28-TW', 'event_ticker...",NaN,NaN
140,KXFRENCHPRES-27,KXFRENCHPRES,In 2027,Who will win the 2027 French presidential elec...,MECNET,True,Elections,"[{'ticker': 'KXFRENCHPRES-27-OFAU', 'event_tic...",NaN,NaN
142,CONTROLS-2026,CONTROLS,In 2026,Which party will win the U.S. Senate in 2026?,MECNET,True,Elections,"[{'ticker': 'CONTROLS-2026-R', 'event_ticker':...",NaN,NaN
143,CONTROLH-2026,CONTROLH,In 2026,Which party will win the House in 2026?,MECNET,True,Elections,"[{'ticker': 'CONTROLH-2026-R', 'event_ticker':...",NaN,NaN


In [13]:
'''
ELECTION KALSHI MARKET
'''
import uuid
import kalshi_python
from kalshi_python.models import *
from pprint import pprint
import os
from dotenv import load_dotenv
import time

load_dotenv()  # Make sure this points to the correct path if .env isn't in the root directory

config = kalshi_python.Configuration()
# Comment the line below to use production
config.host = 'https://api.elections.kalshi.com/trade-api/v2'

# Create an API configuration passing your credentials.
# Use this if you want the kalshi_python sdk to manage the authentication for you.
kalshi_api = kalshi_python.ApiInstance(
    email=os.getenv("EMAIL"),
    password=os.getenv("PASSWORD"),
    configuration=config,
)

# Checks if the exchange is available.
exchangeStatus = kalshi_api.get_exchange_status()
print('Exchange status response: ')
pprint(exchangeStatus)

# Gets the data for a specific series.
seriesTicker = 'KXPRESPARTY'
seriesResponse = kalshi_api.get_series(seriesTicker)
print('\nSeries: ' + seriesTicker)
pprint(seriesResponse)

# Gets the events for a specific series.
seriesTicker = 'KXPRESPARTY'
eventsResponse = kalshi_api.get_events(series_ticker=seriesTicker)
print('\nEvents for series: ' + seriesTicker)
pprint(eventsResponse)

# Gets the data for a specific event.
eventTicker = 'KXPRESPARTY-2028'
eventResponse = kalshi_api.get_event(eventTicker)
print('\nEvent: ' + seriesTicker)
pprint(eventResponse)

marketTicker = 'KXPRESPARTY-2028-R'
marketResponse = kalshi_api.get_market(marketTicker)
print('\nMarket: ' + marketTicker)
pprint(marketResponse)



Exchange status response: 
{'exchange_active': True, 'trading_active': True}

Series: KXPRESPARTY
{'series': {'frequency': 'custom',
            'ticker': 'KXPRESPARTY',
            'title': 'Party winning presidency'}}

Events for series: KXPRESPARTY
{'cursor': '',
 'events': [{'category': 'Elections',
             'event_ticker': 'KXPRESPARTY-2028',
             'mutually_exclusive': True,
             'series_ticker': 'KXPRESPARTY',
             'strike_date': None,
             'strike_period': None,
             'sub_title': 'In 2028',
             'title': 'Which party will win the 2028 Presidential Election?'}]}

Event: KXPRESPARTY
{'event': {'category': 'Elections',
           'event_ticker': 'KXPRESPARTY-2028',
           'mutually_exclusive': True,
           'series_ticker': 'KXPRESPARTY',
           'strike_date': None,
           'strike_period': None,
           'sub_title': 'In 2028',
           'title': 'Which party will win the 2028 Presidential Election?'},
 'markets'